## Fine Tuning

MNISTデータ（手書き数字）のデータを学習したネットワークの結合荷重をFashonMNISTデータ（服装データ）の初期値として利用

- This program was originally written by Mr. Kanda


In [1]:
import numpy as np
import torch

seed = 100
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)

In [2]:
# Dataset : MNIST

from torchvision import datasets, transforms
import torch.utils as utils

transform = transforms.Compose([
    transforms.ToTensor()])

dataset_train_mnist = datasets.MNIST(
    '~/mnist', 
    train=True, 
    download=True, 
    transform=transform)
dataset_test_mnist  = datasets.MNIST(
    '~/mnist', 
    train=False, 
    download=True, 
    transform=transform)

batch_size = 1000

dataloader_train_mnist = utils.data.DataLoader(dataset_train_mnist,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          num_workers=4)
dataloader_test_mnist  = utils.data.DataLoader(dataset_test_mnist,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          num_workers=4)

In [3]:
# Network

import torch.nn as nn
import torch.nn.functional as F
from collections import OrderedDict

class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.conv = nn.Sequential(OrderedDict([
        ("conv1", nn.Conv2d(1, 4, kernel_size=5)),
        ("relu1", nn.ReLU()),
        ("pool1", nn.MaxPool2d(2)),
        ("conv2", nn.Conv2d(4, 8, kernel_size=5)),
        ("relu2", nn.ReLU()),
        ("pool2", nn.MaxPool2d(2)),
    ]))
    self.fc = nn.Sequential(OrderedDict([
        ("fc1"  , nn.Linear(8 * 4 * 4, 100)),
        ("relu1", nn.ReLU()),
        ("fc2"  , nn.Linear(100, 10)),
    ]))

  def forward(self, x1):
    x2 = self.conv(x1)
    x3 = x2.view(x2.size()[0], -1)
    y  = self.fc(x3)
    return y

In [4]:
# Training

from torch import optim

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Net().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

print(model)

for i in range(20):
  print('EPOCH: {epo:}'.format(epo=i+1))

  ### Train ###
  model.train()
  for x, t in dataloader_train_mnist:
    x = x.to(device)
    t = t.to(device)
    model.zero_grad()
    y = model(x)
    loss = criterion(y, t)
    loss.backward()
    optimizer.step()

  ### Evaluate the model by using Training Samples ###
  model.eval()
  sum_loss = 0.0
  sum_correct = 0
  sum_iter = 0
  for x, t in dataloader_train_mnist:
    x = x.to(device)
    t = t.to(device)
    y = model(x)
    loss = criterion(y, t)
    _, predicted = y.max(1)
    sum_loss += loss.cpu().detach().numpy()
    sum_correct += (predicted == t).sum().item()
    sum_iter += 1
  print('  train loss: {loss:.4f},  train acc: {acc:.4f}'.format(loss=sum_loss/sum_iter, acc=sum_correct/(sum_iter*batch_size)))
    
  ### Evalueate the model by using Test Samples ###
  model.eval()
  sum_loss = 0.0
  sum_correct = 0
  sum_iter = 0
  for x, t in dataloader_test_mnist:
    x = x.to(device)
    t = t.to(device)
    y = model(x)
    loss = criterion(y, t)
    _, predicted = y.max(1)
    sum_loss += loss.cpu().detach().numpy()
    sum_correct += (predicted == t).sum().item()
    sum_iter += 1
  print("  test  loss: {loss:.4f},  test  acc: {acc:.4f}".format(loss=sum_loss/sum_iter, acc=sum_correct/(sum_iter*batch_size)))


Net(
  (conv): Sequential(
    (conv1): Conv2d(1, 4, kernel_size=(5, 5), stride=(1, 1))
    (relu1): ReLU()
    (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (conv2): Conv2d(4, 8, kernel_size=(5, 5), stride=(1, 1))
    (relu2): ReLU()
    (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Sequential(
    (fc1): Linear(in_features=128, out_features=100, bias=True)
    (relu1): ReLU()
    (fc2): Linear(in_features=100, out_features=10, bias=True)
  )
)
EPOCH: 1
  train loss: 0.5910,  train acc: 0.8283
  test  loss: 0.5576,  test  acc: 0.8411
EPOCH: 2
  train loss: 0.3224,  train acc: 0.9044
  test  loss: 0.3015,  test  acc: 0.9096
EPOCH: 3
  train loss: 0.2540,  train acc: 0.9242
  test  loss: 0.2363,  test  acc: 0.9289
EPOCH: 4
  train loss: 0.2056,  train acc: 0.9392
  test  loss: 0.1887,  test  acc: 0.9459
EPOCH: 5
  train loss: 0.1769,  train acc: 0.9456
  test  loss: 0.1617,  test  acc: 0.9518
EPO

In [5]:
# Check Parameter before Fine-Tuning

print(model.conv.conv1.weight[0])
print(model.fc.fc2.weight[0])

tensor([[[-0.1009,  0.0867, -0.1843, -0.1771, -0.0883],
         [ 0.1731, -0.0615, -0.1386, -0.0563, -0.0882],
         [-0.0318,  0.2207,  0.2400,  0.2783,  0.1003],
         [ 0.3178,  0.3772,  0.2907,  0.0611,  0.0583],
         [-0.0234,  0.2448,  0.1392,  0.3159,  0.1181]]], device='cuda:0',
       grad_fn=<SelectBackward>)
tensor([ 0.0035,  0.1223, -0.0166, -0.0424,  0.0246, -0.0627,  0.1964, -0.0564,
         0.1239, -0.1341, -0.0689,  0.1357, -0.1843,  0.1066,  0.0121, -0.0744,
         0.0197, -0.1471,  0.1408, -0.0684,  0.0951,  0.0830, -0.0679,  0.0447,
         0.0101,  0.0915, -0.0648, -0.0244, -0.1258, -0.0107,  0.0726,  0.0292,
         0.0286, -0.0902,  0.1043, -0.1635,  0.0574, -0.0191,  0.0493, -0.1651,
         0.0221, -0.0388, -0.0843,  0.0767, -0.1350,  0.1062,  0.0279, -0.1889,
        -0.1058, -0.0326, -0.0434, -0.0508, -0.0491, -0.1057,  0.0470,  0.0383,
        -0.0109,  0.1067, -0.0658, -0.1713, -0.1601, -0.2981,  0.0854, -0.1665,
         0.0974, -0.1156, -0

In [6]:
# Freeze Conv Parameter

for param in model.conv.parameters():
  param.requires_grad = False

for name, param in model.named_parameters():
  print(name, param.requires_grad)

conv.conv1.weight False
conv.conv1.bias False
conv.conv2.weight False
conv.conv2.bias False
fc.fc1.weight True
fc.fc1.bias True
fc.fc2.weight True
fc.fc2.bias True


In [7]:
# Dataset : Fashion-MNIST

from torchvision import datasets, transforms
import torch.utils as utils

transform = transforms.Compose([
    transforms.ToTensor()])

dataset_train_fmnist = datasets.FashionMNIST(
    '~/fashion-mnist', 
    train=True, 
    download=True, 
    transform=transform)
dataset_test_fmnist  = datasets.FashionMNIST(
    '~/fashion-mnist', 
    train=False, 
    download=True, 
    transform=transform)

batch_size = 1000

dataloader_train_fmnist = utils.data.DataLoader(dataset_train_fmnist,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          num_workers=4)
dataloader_test_fmnist  = utils.data.DataLoader(dataset_test_fmnist,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          num_workers=4)

In [8]:
# Fine-Tuning

for i in range(20):
  print(f"EPOCH: {i+1}")

  ### Train ###
  model.train()
  for x, t in dataloader_train_fmnist:
    x = x.to(device)
    t = t.to(device)
    model.zero_grad()
    y = model(x)
    loss = criterion(y, t)
    loss.backward()
    optimizer.step()

  ### Evaluate the model by using Training Samples ###
  model.eval()
  sum_loss = 0.0
  sum_correct = 0
  sum_iter = 0
  for x, t in dataloader_train_fmnist:
    x = x.to(device)
    t = t.to(device)
    y = model(x)
    loss = criterion(y, t)
    _, predicted = y.max(1)
    sum_loss += loss.cpu().detach().numpy()
    sum_correct += (predicted == t).sum().item()
    sum_iter += 1
  print('  train loss: {loss:.4f},  train acc: {acc:.4f}'.format(loss=sum_loss/sum_iter, acc=sum_correct/(sum_iter*batch_size)))    

  ### Evalueae the model by using Test Samples ###
  model.eval()
  sum_loss = 0.0
  sum_correct = 0
  sum_iter = 0
  for x, t in dataloader_test_fmnist:
    x = x.to(device)
    t = t.to(device)
    y = model(x)
    loss = criterion(y, t)
    _, predicted = y.max(1)
    sum_loss += loss.cpu().detach().numpy()
    sum_correct += (predicted == t).sum().item()
    sum_iter += 1
  print("  test  loss: {loss:.4f},  test  acc: {acc:.4f}".format(loss=sum_loss/sum_iter, acc=sum_correct/(sum_iter*batch_size)))


EPOCH: 1
  train loss: 0.6808,  train acc: 0.7463
  test  loss: 0.7037,  test  acc: 0.7376
EPOCH: 2
  train loss: 0.6238,  train acc: 0.7682
  test  loss: 0.6488,  test  acc: 0.7555
EPOCH: 3
  train loss: 0.5850,  train acc: 0.7847
  test  loss: 0.6106,  test  acc: 0.7735
EPOCH: 4
  train loss: 0.5630,  train acc: 0.7921
  test  loss: 0.5877,  test  acc: 0.7831
EPOCH: 5
  train loss: 0.5498,  train acc: 0.8004
  test  loss: 0.5772,  test  acc: 0.7889
EPOCH: 6
  train loss: 0.5352,  train acc: 0.8056
  test  loss: 0.5619,  test  acc: 0.7957
EPOCH: 7
  train loss: 0.5220,  train acc: 0.8105
  test  loss: 0.5498,  test  acc: 0.8025
EPOCH: 8
  train loss: 0.5189,  train acc: 0.8083
  test  loss: 0.5468,  test  acc: 0.7964
EPOCH: 9
  train loss: 0.5133,  train acc: 0.8143
  test  loss: 0.5431,  test  acc: 0.8030
EPOCH: 10
  train loss: 0.4990,  train acc: 0.8177
  test  loss: 0.5271,  test  acc: 0.8090
EPOCH: 11
  train loss: 0.4927,  train acc: 0.8210
  test  loss: 0.5204,  test  acc: 0.81

In [9]:
# Check Parameter after Fine-Tuning

print(model.conv.conv1.weight[0])
print(model.fc.fc2.weight[0])

tensor([[[-0.0987,  0.0879, -0.1839, -0.1773, -0.0885],
         [ 0.1744, -0.0606, -0.1380, -0.0560, -0.0881],
         [-0.0309,  0.2215,  0.2408,  0.2791,  0.1009],
         [ 0.3180,  0.3777,  0.2913,  0.0617,  0.0589],
         [-0.0238,  0.2449,  0.1394,  0.3165,  0.1186]]], device='cuda:0')
tensor([ 0.0733,  0.0787, -0.1068, -0.0136,  0.0326, -0.0627,  0.2522, -0.0872,
         0.0572, -0.0669, -0.4504,  0.1909, -0.3029,  0.2289, -0.0150, -0.0468,
         0.0593, -0.1911,  0.0770, -0.0960,  0.0730,  0.0593, -0.0585,  0.0385,
         0.0283,  0.1394, -0.0180, -0.0244, -0.2142,  0.0117,  0.2173, -0.0178,
         0.0679, -0.0951,  0.1260, -0.2009,  0.0811, -0.0177,  0.0493, -0.0703,
         0.0029,  0.0346, -0.0475,  0.0728, -0.1105,  0.2052,  0.0417, -0.1982,
        -0.0499,  0.0131, -0.0183, -0.0523, -0.0470, -0.0475,  0.0154,  0.0383,
        -0.0412,  0.1126, -0.0658, -0.1894, -0.1234, -0.2858,  0.0934, -0.1142,
         0.1552, -0.0764,  0.0054,  0.0426, -0.1289,  0.1251,

参考<br>
https://pytorch.org/docs/stable/notes/autograd.html